In [1]:
import sys
sys.path.append('../../..')
from lib import automated_evaluation, chat_helper
from anthropic import Anthropic
from lib.automated_evaluation import  caesar_decrypt
from lib.hallucination_lib import get_letter_probabilities_batched
from transformers import AutoTokenizer
import pickle
import pandas as pd
from tqdm import tqdm
import torch
token = 'redacted_for_publishing'
import os
import re
import json
import matplotlib.pyplot as plt
import numpy as np
system_prompt = "You are a helpful, honest and concise assistant."  # Needed for chat model


In [2]:
def rate_all_answers(questions):
    for key in questions.keys():
        if "truth_answer" in key:
            i = int(re.match(r"truth_answer_(\d+)$", key).group(1))
            answer_name = "truth_rating_" + str(i)
            if answer_name not in questions.keys():
                rate_df = pd.DataFrame(
                    {
                        "questions": questions["truth_question"].tolist(),
                        "answers": questions[key].tolist(),
                        "solutions": questions["truth"].tolist(),
                    }
                )
                automated_evaluation.rate_answers_to_true_questions(rate_df)
                questions[answer_name] = rate_df["rating"].tolist()
                questions["truth_rating_text_" + str(i)] = rate_df[
                    "rating_text"
                ].tolist()
        if "fiction_answer" in key:
            i = int(re.match(r"fiction_answer_(\d+)$", key).group(1))
            answer_name = "fiction_rating_" + str(i)
            if answer_name not in questions.keys():
                rate_df = pd.DataFrame(
                    {
                        "question": questions["fiction_question"].tolist(),
                        "answers": questions[key].tolist(),
                        "fiction": questions["fiction"].tolist(),
                        "truth": questions["truth"].tolist(),
                    }
                )
                automated_evaluation.classify_affirmation_answer(rate_df)
                questions[answer_name] = rate_df["rating"].tolist()
                questions["fiction_rating_text_" + str(i)] = rate_df[
                    "rating_text"
                ].tolist()


In [4]:
data_path  = "./steered_completions/"
for folder in os.listdir(data_path):
    for file in os.listdir(data_path+folder):
        path = data_path+folder+"/"+file
        print(path)
        data = pd.read_csv(path)
        rate_all_answers(data)
        data.to_csv(path)

./steered_completions/alluding_questions/alluding_questions_steered.csv
./steered_completions/alluding_questions/conversation_steered.csv
./steered_completions/alluding_questions/direct_questions_steered.csv
./steered_completions/alluding_questions/questioning_assuming_statement_steered.csv
./steered_completions/alluding_questions/unsteered.csv
./steered_completions/conversation/alluding_questions_steered.csv
./steered_completions/conversation/conversation_steered.csv
./steered_completions/conversation/direct_questions_steered.csv
./steered_completions/conversation/questioning_assuming_statement_steered.csv
./steered_completions/conversation/unsteered.csv
./steered_completions/direct_questions/alluding_questions_steered.csv
./steered_completions/direct_questions/conversation_steered.csv
./steered_completions/direct_questions/direct_questions_steered.csv
./steered_completions/direct_questions/questioning_assuming_statement_steered.csv
./steered_completions/direct_questions/unsteered.csv